# Loop 3 Analysis: Fixing Overlap Validation

The submission failed with 'Overlapping trees in group 002'. Need to understand why my strict validation isn't catching what Kaggle's validator catches.

In [1]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
import os

# Set precision for Decimal (matching official kernel)
getcontext().prec = 25
scale_factor = Decimal('1e15')

# Tree vertices (from official kernel)
TX = [0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125]
TY = [0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5]

In [2]:
def make_tree_polygon_official(center_x, center_y, angle):
    """Create tree polygon using official method with Decimal precision"""
    trunk_w = Decimal('0.15')
    trunk_h = Decimal('0.2')
    base_w = Decimal('0.7')
    mid_w = Decimal('0.4')
    top_w = Decimal('0.25')
    tip_y = Decimal('0.8')
    tier_1_y = Decimal('0.5')
    tier_2_y = Decimal('0.25')
    base_y = Decimal('0.0')
    trunk_bottom_y = -trunk_h

    initial_polygon = Polygon(
        [
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ]
    )
    rotated = affinity.rotate(initial_polygon, float(angle), origin=(0, 0))
    translated = affinity.translate(rotated,
                                    xoff=float(Decimal(str(center_x)) * scale_factor),
                                    yoff=float(Decimal(str(center_y)) * scale_factor))
    return translated

In [3]:
def check_overlaps_official(xs, ys, degs):
    """Check overlaps using official method: intersects but not touches"""
    n = len(xs)
    if n <= 1:
        return True, []
    
    polygons = [make_tree_polygon_official(xs[i], ys[i], degs[i]) for i in range(n)]
    
    overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    
    return len(overlaps) == 0, overlaps

In [4]:
# Load the failed submission
failed_path = '/home/code/submission_candidates/candidate_002.csv'
df = pd.read_csv(failed_path)

# Parse values
df['x_val'] = df['x'].str.replace('s', '').astype(float)
df['y_val'] = df['y'].str.replace('s', '').astype(float)
df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
df['N'] = df['id'].str.split('_').str[0].astype(int)

print(f"Loaded {len(df)} rows")
print(f"N values: {df['N'].nunique()}")

# Check N=2 specifically (the failing group)
n2_data = df[df['N'] == 2]
print(f"\nN=2 data:")
print(n2_data)

Loaded 20100 rows
N values: 200

N=2 data:
      id           x           y          deg     x_val     y_val     deg_val  \
1  002_0   s0.154097  s-0.038541  s203.629378  0.154097 -0.038541  203.629378   
2  002_1  s-0.154097  s-0.561459   s23.629378 -0.154097 -0.561459   23.629378   

   N  
1  2  
2  2  


In [5]:
# Check N=2 for overlaps using official method
xs = n2_data['x_val'].values
ys = n2_data['y_val'].values
degs = n2_data['deg_val'].values

print(f"N=2 coordinates:")
for i in range(len(xs)):
    print(f"  Tree {i}: x={xs[i]:.12f}, y={ys[i]:.12f}, deg={degs[i]:.12f}")

valid, overlaps = check_overlaps_official(xs, ys, degs)
print(f"\nN=2 valid (official method): {valid}")
print(f"Overlaps: {overlaps}")

N=2 coordinates:
  Tree 0: x=0.154097000000, y=-0.038541000000, deg=203.629378000000
  Tree 1: x=-0.154097000000, y=-0.561459000000, deg=23.629378000000

N=2 valid (official method): False
Overlaps: [(0, 1)]


In [6]:
# Compare with baseline N=2
baseline_path = '/home/code/preoptimized/submission.csv'
baseline_df = pd.read_csv(baseline_path)
baseline_df['x_val'] = baseline_df['x'].str.replace('s', '').astype(float)
baseline_df['y_val'] = baseline_df['y'].str.replace('s', '').astype(float)
baseline_df['deg_val'] = baseline_df['deg'].str.replace('s', '').astype(float)
baseline_df['N'] = baseline_df['id'].str.split('_').str[0].astype(int)

baseline_n2 = baseline_df[baseline_df['N'] == 2]
print("Baseline N=2:")
print(baseline_n2)

xs_b = baseline_n2['x_val'].values
ys_b = baseline_n2['y_val'].values
degs_b = baseline_n2['deg_val'].values

valid_b, overlaps_b = check_overlaps_official(xs_b, ys_b, degs_b)
print(f"\nBaseline N=2 valid: {valid_b}")
print(f"Overlaps: {overlaps_b}")

Baseline N=2:
      id                     x                      y                  deg  \
1  002_0   s0.1540970696213643  s-0.03854074269478543  s203.62937773065684   
2  002_1  s-0.1540970696213643   s-0.5614592573052146  s23.629377730656792   

      x_val     y_val     deg_val  N  
1  0.154097 -0.038541  203.629378  2  
2 -0.154097 -0.561459   23.629378  2  

Baseline N=2 valid: True
Overlaps: []


In [ ]:
# Check ALL N values in failed submission for overlaps
print("Checking all N values for overlaps...")
failing_groups = []

for n in range(1, 201):
    n_data = df[df['N'] == n]
    xs = n_data['x_val'].values
    ys = n_data['y_val'].values
    degs = n_data['deg_val'].values
    
    valid, overlaps = check_overlaps_official(xs, ys, degs)
    if not valid:
        failing_groups.append((n, len(overlaps)))

print(f"\nFailing groups: {len(failing_groups)}")
for n, num_overlaps in failing_groups[:20]:
    print(f"  N={n}: {num_overlaps} overlaps")

In [ ]:
# The issue is clear: precision truncation!
# Baseline has full precision: s0.1540970696213643
# My submission has truncated: s0.154097

# Let me check the raw values in both files
print("Baseline raw values:")
print(baseline_df[baseline_df['N'] == 2][['x', 'y', 'deg']])

print("\nFailed submission raw values:")
print(df[df['N'] == 2][['x', 'y', 'deg']])

# The problem is that when I generated the submission, I truncated the precision
# This causes near-touching trees to overlap